In [1]:
import os
import re
import itertools
import pandas as pd
from sklearn.metrics import cohen_kappa_score as cks
from tabulate import tabulate

### Building the dataframe with the results

In [2]:
datapath = 'E:\\IRELAND\\ChiA\\Chia\\data\\img_labeling_2nd_round\\'

In [4]:
label_files = sorted(os.listdir(datapath))
label_files = [f for f in label_files if 'A00' in f]
for f in label_files:
    print(f)

1st_task_labeling_A001_2020-11-04-07-41-41.csv
1st_task_labeling_A002_2020-10-23-12-56-45.csv
1st_task_labeling_A003_2020-11-09-04-46-17.csv
1st_task_labeling_A004_2020-11-06-04-46-17.csv
1st_task_labeling_A005_2020-11-02-01-40-27.csv
1st_task_labeling_A006_2020-11-04-07-41-41.csv
2nd_task_labeling_A001_2020-11-04-09-14-46.csv
2nd_task_labeling_A002_2020-10-29-10-18-19.csv
2nd_task_labeling_A003_2020-11-09-04-46-17.csv
2nd_task_labeling_A004_2020-11-09-10-31-33.csv
2nd_task_labeling_A005_2020-11-02-01-40-27.csv
2nd_task_labeling_A006_2020-11-04-07-41-41.csv
3rd_task_labeling_A001_2020-11-04-08-14-15.csv
3rd_task_labeling_A002_2020-10-29-10-37-58.csv
3rd_task_labeling_A003_2020-11-09-04-46-17.csv
3rd_task_labeling_A004_2020-11-09-11-51-56.csv
3rd_task_labeling_A005_2020-11-02-01-40-27.csv
3rd_task_labeling_A006_2020-11-04-07-41-41.csv


In [7]:
list_dfs = []
for label_file in label_files:
    task = int(re.findall(r'[1-3]{1}', label_file)[0])
    user = re.findall(r'A001|A002|A003|A004|A005|A006', label_file)[0]
    date = re.findall(r'202[0-1]{1}.[0-9]{2}.[0-9]{2}', label_file)[0]
    #date = f'{date}-2020'
    
    df = pd.read_csv(f'{datapath}{label_file}', names=['image name', 'class','w','h'])
    df['task'] = task
    df['user'] = user
    df['date'] = date
    df['date'] = pd.to_datetime(df['date'])
    df.drop(['w','h'], axis=1, inplace=True)
    df.drop_duplicates(inplace=True)
    df = df[['task', 'user', 'date', 'image name', 'class']]
    print(f'Task: {task}, User: {user}, Images Classified: {len(df)}')
    list_dfs.append(df)

Task: 1, User: A001, Images Classified: 392
Task: 1, User: A002, Images Classified: 392
Task: 1, User: A003, Images Classified: 392
Task: 1, User: A004, Images Classified: 392
Task: 1, User: A005, Images Classified: 392
Task: 1, User: A006, Images Classified: 392
Task: 2, User: A001, Images Classified: 392
Task: 2, User: A002, Images Classified: 392
Task: 2, User: A003, Images Classified: 392
Task: 2, User: A004, Images Classified: 392
Task: 2, User: A005, Images Classified: 392
Task: 2, User: A006, Images Classified: 392
Task: 3, User: A001, Images Classified: 392
Task: 3, User: A002, Images Classified: 442
Task: 3, User: A003, Images Classified: 392
Task: 3, User: A004, Images Classified: 392
Task: 3, User: A005, Images Classified: 392
Task: 3, User: A006, Images Classified: 392


In [8]:
df_labeling = pd.concat(list_dfs)
df_labeling['id_image'] = pd.Categorical(df_labeling['image name']).codes
df_labeling["class"] = df_labeling["class"].apply(lambda x:x[1:-1])
df_labeling = df_labeling[['task', 'user', 'date', 'image name', 'id_image', 'class']]

In [9]:
df_labeling.head()

,task,user,date,image name,id_image,class
0,1,A001,2020-11-04,203-resource_document_zuiderzeemuseum_B001601_...,99,Non-Fruits
1,1,A001,2020-11-04,285-gam19649_1.jpeg,282,Fruits
2,1,A001,2020-11-04,07101-O_389_1.jpeg,6,Fruits
3,1,A001,2020-11-04,07101-O_927_1.jpeg,12,Fruits
4,1,A001,2020-11-04,07101-O_957_1.jpeg,13,Fruits


In [10]:
df_labeling['class'].unique()

array(['Non-Fruits', 'Fruits', 'Formal', 'Informal', 'Appealing',
       'Non-appealing'], dtype=object)

In [11]:
df_labeling['image_appearances'] = df_labeling.groupby('id_image')['id_image'].transform('count')
df_labeling = df_labeling[df_labeling['image_appearances'] != 1] 

In [12]:
df_labeling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7056 entries, 0 to 391
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   task               7056 non-null   int64         
 1   user               7056 non-null   object        
 2   date               7056 non-null   datetime64[ns]
 3   image name         7056 non-null   object        
 4   id_image           7056 non-null   int16         
 5   class              7056 non-null   object        
 6   image_appearances  7056 non-null   int64         
dtypes: datetime64[ns](1), int16(1), int64(2), object(3)
memory usage: 399.7+ KB


In [13]:
df_labeling.head()

,task,user,date,image name,id_image,class,image_appearances
0,1,A001,2020-11-04,203-resource_document_zuiderzeemuseum_B001601_...,99,Non-Fruits,18
1,1,A001,2020-11-04,285-gam19649_1.jpeg,282,Fruits,18
2,1,A001,2020-11-04,07101-O_389_1.jpeg,6,Fruits,18
3,1,A001,2020-11-04,07101-O_927_1.jpeg,12,Fruits,18
4,1,A001,2020-11-04,07101-O_957_1.jpeg,13,Fruits,18


#### Checking the numbers of images classified after fixing names:

In [14]:
for task in df_labeling.task.unique():
    for user in df_labeling.user.unique():
        size = len(df_labeling.loc[(df_labeling.user == user) & (df_labeling.task == task), 'id_image'].unique())
        print(f'Task: {task}, User: {user}, Images Classified: {size}')
    print()

Task: 1, User: A001, Images Classified: 392
Task: 1, User: A002, Images Classified: 392
Task: 1, User: A003, Images Classified: 392
Task: 1, User: A004, Images Classified: 392
Task: 1, User: A005, Images Classified: 392
Task: 1, User: A006, Images Classified: 392

Task: 2, User: A001, Images Classified: 392
Task: 2, User: A002, Images Classified: 392
Task: 2, User: A003, Images Classified: 392
Task: 2, User: A004, Images Classified: 392
Task: 2, User: A005, Images Classified: 392
Task: 2, User: A006, Images Classified: 392

Task: 3, User: A001, Images Classified: 392
Task: 3, User: A002, Images Classified: 392
Task: 3, User: A003, Images Classified: 392
Task: 3, User: A004, Images Classified: 392
Task: 3, User: A005, Images Classified: 392
Task: 3, User: A006, Images Classified: 392



#### Checking which images were not classified:

In [15]:
images = set(df_labeling.id_image.unique())
dfs = []
for task in df_labeling.task.unique():
    for user in df_labeling.user.unique():
        id_list = []
        ids = images.difference(set(df_labeling.loc[(df_labeling.user == user) & (df_labeling.task == task), 'id_image']))
        if len(ids) > 0:
            #print(f'Missing files for task: {task}, by user: {user}')
            for _id in ids:
                try:
                    id_list.append(df_labeling.loc[(df_labeling.task == task) & (df_labeling.id_image == _id), 'image name'].values[0])
                    #print(df_labeling.loc[(df_labeling.task == task) & (df_labeling.id_image == _id), 'image name'].values[0])
                except:
                    pass
                    #print('--->', df_labeling.loc[(df_labeling.task == task) & (df_labeling.id_image == _id), 'image name'].values)
            #print()
        dfm = pd.DataFrame({'images':id_list})
        dfm['task'] = task
        dfm['user'] = user
        dfs.append(dfm)
df_missing = pd.concat(dfs)
df_missing.to_excel(f'{datapath}/missing.xlsx')
df_missing.head()       

,images,task,user


#### Checking duplicates

In [16]:
duplicates = df_labeling.loc[df_labeling.duplicated(['task', 'user', 'image name', 'id_image'], keep=False), ['task', 'user', 'image name', 'id_image', 'class']]
duplicates.sort_values(by=['user', 'image name', 'task'], inplace=True)
duplicates.to_excel(f'{datapath}/duplicates.xlsx')
duplicates.head(30)

,task,user,image name,id_image,class


### Analysing the [inter-annotator agreement](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.cohen_kappa_score.html) on the results

In [17]:
l1 = df_labeling.user.unique()
iter_users = list(itertools.product(l1,l1))
df_iaa = pd.DataFrame(index=l1, columns=l1)

In [18]:
for task in df_labeling.task.unique():
    total_agreement = 0
    for user1,user2 in iter_users:
        classesA = df_labeling.loc[(df_labeling.user == user1) & (df_labeling.task == task),['id_image', 'class']]
        classesA.sort_values(by=['id_image'], inplace=True)

        classesB = df_labeling.loc[(df_labeling.user == user2) & (df_labeling.task == task),['id_image', 'class']]
        classesB.sort_values(by=['id_image'], inplace=True)
        
        classesAB = pd.merge(classesA, classesB, on=['id_image'])
        classesAB.drop_duplicates(subset='id_image', keep = 'first', inplace=True) 
        classesAB.drop('id_image', axis=1, inplace=True)
        classesAB.dropna(inplace=True)

        agreement = cks(classesAB['class_x'], classesAB['class_y'])
        total_agreement += agreement
        df_iaa.loc[user1,user2] = f'{agreement:.3f}/({len(classesAB)})'  
        df_iaa.index.name = f'Task_{task}'
    print(tabulate(df_iaa, headers='keys', tablefmt='psql'))
    print(f'\nThe average agreement was {total_agreement/(len(l1)**2):.3f}\n\n')

+----------+-------------+-------------+-------------+-------------+-------------+-------------+
| Task_1   | A001        | A002        | A003        | A004        | A005        | A006        |
|----------+-------------+-------------+-------------+-------------+-------------+-------------|
| A001     | 1.000/(392) | 0.943/(392) | 0.913/(392) | 0.928/(392) | 0.913/(392) | 0.912/(392) |
| A002     | 0.943/(392) | 1.000/(392) | 0.886/(392) | 0.912/(392) | 0.866/(392) | 0.865/(392) |
| A003     | 0.913/(392) | 0.886/(392) | 1.000/(392) | 0.923/(392) | 0.928/(392) | 0.918/(392) |
| A004     | 0.928/(392) | 0.912/(392) | 0.923/(392) | 1.000/(392) | 0.913/(392) | 0.881/(392) |
| A005     | 0.913/(392) | 0.866/(392) | 0.928/(392) | 0.913/(392) | 1.000/(392) | 0.897/(392) |
| A006     | 0.912/(392) | 0.865/(392) | 0.918/(392) | 0.881/(392) | 0.897/(392) | 1.000/(392) |
+----------+-------------+-------------+-------------+-------------+-------------+-------------+

The average agreement was 0.9

In [17]:
df_labeling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7056 entries, 0 to 391
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   task               7056 non-null   int64         
 1   user               7056 non-null   object        
 2   date               7056 non-null   datetime64[ns]
 3   image name         7056 non-null   object        
 4   id_image           7056 non-null   int16         
 5   class              7056 non-null   object        
 6   image_appearances  7056 non-null   int64         
dtypes: datetime64[ns](1), int16(1), int64(2), object(3)
memory usage: 399.7+ KB


In [17]:
df_labeling['round'] = 2
df_labeling.drop('image_appearances', inplace=True, axis=1)

In [18]:
#df_labeling.to_hdf('../data/df_labeling.hdf', key='round2')